- Repeat BaseLine_Model with normalised data.
    - Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.
- How does the mean of the mean squared errors compare to that from BaseLine_Model?

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [4]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
# find some quick useful statistics
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [7]:
#Split the data into predictor and target variables for Regression model
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column
X = predictors
y = target

In [8]:
n_cols = X.shape[1]
n_cols

8

- Make Normalization

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [21]:
X_norm = predictors_norm
X_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [11]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

- Split the data into training and test sets with scikitlearn

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.30, random_state=42)

- Build the neural network, train using 50 epochs

In [23]:
# define regression model
def regression_model():
    # Construct the Network 
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [24]:
model = regression_model()
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
23/23 [==============================] - 1s 2ms/step - loss: 1602.1824
Epoch 2/50
23/23 [==============================] - 0s 3ms/step - loss: 1584.9337
Epoch 3/50
23/23 [==============================] - 0s 3ms/step - loss: 1567.8754
Epoch 4/50
23/23 [==============================] - 0s 3ms/step - loss: 1550.9735
Epoch 5/50
23/23 [==============================] - 0s 3ms/step - loss: 1533.7020
Epoch 6/50
23/23 [==============================] - 0s 5ms/step - loss: 1516.3132
Epoch 7/50
23/23 [==============================] - 0s 4ms/step - loss: 1498.3481
Epoch 8/50
23/23 [==============================] - 0s 3ms/step - loss: 1479.5599
Epoch 9/50
23/23 [==============================] - 0s 5ms/step - loss: 1460.1641
Epoch 10/50
23/23 [==============================] - 0s 3ms/step - loss: 1439.9386
Epoch 11/50
23/23 [==============================] - 0s 3ms/step - loss: 1418.6423
Epoch 12/50
23/23 [==============================] - 0s 3ms/step - loss: 1396.0815
Epoch 13/50
2

In [25]:
#Model evaulation on tested data
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 2ms/step - loss: 320.7303


320.73028564453125

In [26]:
y_pred = model.predict(X_test)
y_pred

10/10 [==============================] - 0s 5ms/step


array([[50.225864 ],
       [25.858511 ],
       [46.13838  ],
       [25.653313 ],
       [20.867226 ],
       [18.196274 ],
       [10.385321 ],
       [40.844208 ],
       [32.589622 ],
       [19.268534 ],
       [17.99307  ],
       [12.657227 ],
       [73.113914 ],
       [23.333244 ],
       [23.135962 ],
       [24.497513 ],
       [10.948334 ],
       [17.779753 ],
       [12.1731205],
       [12.772827 ],
       [18.983683 ],
       [24.586044 ],
       [27.811064 ],
       [18.310383 ],
       [25.763617 ],
       [36.112362 ],
       [13.557608 ],
       [20.547789 ],
       [33.030704 ],
       [10.980091 ],
       [30.886496 ],
       [16.932781 ],
       [32.217083 ],
       [37.931652 ],
       [11.9381485],
       [28.140423 ],
       [30.950266 ],
       [17.883348 ],
       [ 6.4419484],
       [22.75592  ],
       [16.092388 ],
       [ 5.9785504],
       [12.983282 ],
       [43.19127  ],
       [ 9.336441 ],
       [27.268627 ],
       [50.079674 ],
       [38.83

In [27]:
MSE1 = mean_squared_error(y_test, y_pred)
MSE1

320.7302794974185

In [28]:
mean1 = np.mean(MSE1)
sd1 = np.std(MSE1)
print(mean1, sd1)

320.7302794974185 0.0


- create a list of 50 mean squared errors.

In [29]:
list1_MSE = []
for i in tqdm(range (0, 50)):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.30, random_state=i)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    value_MSE1 = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    MSE1 = mean_squared_error(y_test, y_pred)
    list1_MSE.append(MSE1)
    print("MSE" + str(i+1) + "=" + str(value_MSE1))

mean1 = np.mean(list1_MSE)
sd1 = np.std(list1_MSE)

print("Mean: "+str(mean1))
print("Standard Deviation: "+str(sd1))

  0%|          | 0/50 [00:00<?, ?it/s]

10/10 [==============================] - 0s 2ms/step


  2%|▏         | 1/50 [00:01<01:33,  1.91s/it]

MSE1=145.48773193359375
10/10 [==============================] - 0s 1ms/step


  4%|▍         | 2/50 [00:04<01:37,  2.03s/it]

...

MSE48=33.78548812866211
10/10 [==============================] - 0s 1ms/step


 98%|█████████▊| 49/50 [01:38<00:01,  1.83s/it]

MSE49=33.50639724731445
10/10 [==============================] - 0s 1ms/step


100%|██████████| 50/50 [01:39<00:00,  2.00s/it]

MSE50=36.516666412353516
Mean: 42.26450219574748
Standard Deviation: 21.512701293692267


## Conclusion
- The mean squared errors of the list of 50 MSEs that were generated from 50 repetitions of running a neural network model based on regression with Normalization is 42.2 which is less than base lile 51. Normalising the predictors improves the mean squared errors by decreasing the total value.